In [ ]:
import numpy as np
from scipy.sparse import load_npz, lil_matrix, identity, csr_matrix
from sklearn.preprocessing import normalize
import gzip
import json
from tqdm.notebook import tqdm

sl = load_npz('data/graph4sparse_1.npz')
print(sl.count_nonzero(), sl.max(), sl.min())


In [ ]:
if False:
    fi = gzip.open('data/wikidata_translation_v1.tsv.gz', 'rb')
    j = json.load(open(f'data/label4sparse.json'))
    QUS = int(j['maxq'])

    bigq = np.zeros((QUS, 200), dtype=np.half)
    print(fi.readline())
    uk = 0
    for l in tqdm(fi, total=143_121_769):
        l = l.decode().split('\t')
        if '<http://www.wikidata.org/entity/Q' in l[0]:
            try:
                qid = int(l[0].replace('<http://www.wikidata.org/entity/Q', '').replace('>', ''))
                bigq[qid] = [np.half(a) for a in l[1:]]
                uk += 1
            except Exception as e:
                pass
    np.savez_compressed('data/bigq', vectors=bigq)        
    print(uk)
else:
    bigq = np.load('data/bigq.npz')['vectors']
print(bigq.shape)

In [ ]:
if False:
    qrank = np.zeros((1152625061), int)
    fi = gzip.open('/backup/wikidata/qrank.csv.gz', 'rb')
    fi.readline()
    uk = 0
    for l in fi:
        q, r = l.decode().strip('\n').split(',')
        qrank[int(q[1:])] = int(r)
        uk += 1
    qrank = np.log(qrank+.0001)
    qrank = (qrank / qrank.max()).astype(np.half)
    print(uk)#, print([qrank[int(q[1:])] for q in qids])
    np.savez_compressed('data/qrank', vectors=qrank)        
else:
    qrank = np.load('data/qrank.npz')['vectors']
print(qrank.shape)

In [ ]:
#!pip install tensorflow_hub
import tensorflow as tf
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
model = hub.load(module_url)
m = model(['python programmer', 'python snake', 'computer science', 'biology', 'c++ programming language'])
np.inner(m, m)

In [ ]:
qids = ['Q28865', 'Q271218', 'Q10811', 'Q2407', 'Q842014', 'Q21198', 'Q420', 'Q80006', 'Q460387', 'Q179012']
desc = ['Python language', 'Python snake', 'Reptils', 'C++', 'Django Python', 'Computer Science', 'Biology', 'Computer Programming', 'Scrum', 'Project management']

qs = [int(a[1:]) for a in qids]

In [ ]:
#%%timeit
#sl[qs].todense()[:,qs]
#m = normalize(sl[qs], norm='l2', axis=1)
m = sl[qs]
slic = m*m.T
slic.data = np.minimum(slic.data / 5., 1)

slic = (slic / slic.max()) + m.todense()[:,qs]
#slic = m.todense()[:,qs]

for i in range(len(qs)):
    for j in range(len(qs)):
        if i <= j:
            print(desc[i], desc[j], qrank[qs[j]], qrankl[qs[j]], round(slic[i, j], 2))
    

In [ ]:
m = csr_matrix([[0,0,0],[1,0,0],[0,1,0],])
m = m*m*0.8
m.todense()

In [ ]:
from wiki_labels import find_qid, qid_en_desc_get, qid_lab_get, descl, tried
tried['common name of Homo sapiens, unique extant species of the genus Homo']

In [3]:
qid_en_desc_get(42)

'English science fiction writer and humourist (1952–2001)'